In [ ]:
!pip install seaborn

You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import sys
from enum import Enum
from sympy import *

pd.options.display.max_columns = 200

from matplotlib import rcParams
rcParams['figure.figsize'] = 14, 12
rcParams['font.size'] = 16
rcParams['axes.labelsize'] = 14
rcParams['xtick.labelsize'] = 13
rcParams['ytick.labelsize'] = 13
rcParams['legend.fontsize'] = 15

import seaborn as sns
sns.set_style("whitegrid")

In [ ]:
!pip install sympy

You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.


## Метод деления отрезка пополам

In [4]:
def bisection_method(a, b, x, f, r, eps=0.001):
    delta = 0.0001
    n = 0
    while abs(b-a) >= eps:
        n += 1
        u1 = (b + a - delta)/2
        u2 = (b + a + delta)/2
        f_u1 = f(u1, r)
        f_u2 = f(u2, r)
        if f_u1 < f_u2:
            b = u2
        elif f_u1 > f_u2:
            a = u1
        elif f_u1 == f_u2:
            b = u2
            a = u1
        # print(a, b)
    return (b+a)/2, f((b+a)/2, r), n

## Метод "золотого сечения"

In [0]:
def golden_section(f, a, b, eps):
    n = 0
    # print((" {0:.8s}  ||   {1:.1s}    ||    {2:.1s}   || {3:.5s}  ||   {4:.2s}   ||   {5:.2s}   ||  {6:.5s}  ||  {7:.5s}").format("Итерация", "a", "b", "b - a", "u1", "u2", "J(u1)", "J(u2)"))
    alpha = (math.sqrt(5) - 1) / 2
    alpha_1 = (3 - math.sqrt(5)) / 2
    u_1 = a + alpha_1 * (b-a)
    u_2 = a + alpha * (b-a)
    J_1 = f(u_1)
    J_2 = f(u_2)
    while b - a >= eps:
        n += 1
        # print(("     {0:.0f}     || {1:.4f} || {2:.4f} || {3:.4f} || {4:.4f} || {5:.4f} || {6:.4f} || {7:.4f}").format(n,a, b, b - a, u_1, u_2, J_1, J_2))
        if J_1 < J_2:
            b = u_2
            u_2 = u_1
            J_2 = J_1
            u_1 = a + alpha_1 * (b-a)
            J_1 = f(u_1)
        elif J_1 > J_2:
            a = u_1
            u_1 = u_2
            J_1 = J_2
            u_2 = a + alpha * (b-a)
            J_2 = f(u_2)
        else:
            b = u_2
            a = u_1
            u_1 = a + alpha_1 * (b-a)
            u_2 = a + alpha * (b-a)
            J_1 = f(u_1)
            J_2 = f(u_2)
    n = n+1
    # print(("     {0:.0f}     || {1:.4f} || {2:.4f} || {3:.4f} || {4:.4f} || {5:.4f} || {6:.4f} || {7:.4f}").format(n,a, b, b - a, u_1, u_2, J_1, J_2))
    u_min = (b + a) / 2
    J_min = f(u_min)
    return u_min, J_min, n

## Метод парабол

In [0]:
def f_parabola(u1, u3, x, f, eps):
    u2 = (u3 + u1) / 2
    i = 0
    u2_test = u2
    delta_minus = f(u1) - f(u2)
    delta_plus = f(u3) - f(u2)
    w = u2 + ((u3 - u2)**2 * delta_minus - (u2 - u1)**2 * delta_plus)/(2*((u3 - u2) * delta_minus + (u2 - u1) * delta_plus))
    while abs(u2_test - w) >= eps:
        i += 1
        delta_minus = f(u1) - f(u2)
        delta_plus = f(u3) - f(u2)
        if delta_minus + delta_plus == 0:
            sys.exit('strange')
        try:
            w = u2 + (((u3 - u2)**2 * delta_minus - (u2 - u1)**2 * delta_plus)/(2*((u3 - u2) * delta_minus + (u2 - u1) * delta_plus)))
        except ZeroDivisionError as excep:
            return u2, f(u2)
        u2_test = u2
        # print(f"u1={u1}, u2={u2}, u3={u3}")
        if w < u2:
            if f(w) < f(u2):
                # print(1)
                u3 = u2
                u2 = w
            elif f(w) > f(u2):
                # print(2)
                u1 = w
            else:
                # print(3)
                if f(u1) > f(u2):
                    u3 = u2
                    u2 = w
                elif f(u2) > f(u3):
                    u1 = w
        elif w > u2:
            if f(w) < f(u2):
                # print(4)
                u1 = u2
                u2 = w
            elif f(w) > f(u2):
                # print(5)
                u3 = w
            else:
                if f(u3) > f(u2):
                    u1 = u2
                    u2 = w
                elif f(u1) > f(u2):
                    u3 = w
        else:
            if f(u2 - 0.01) < f(u2):
                u2 -= 0.01
            else:
                u2 += 0.01
    return u2, f(u2), i

## Метод Ньютона

In [0]:
def newton(u0, defaults, eps):
    def f_diff_1(x):
        return defaults[0]*5 * x**4 + defaults[1]*4 * x**3 + defaults[2]*3 * x**2 + defaults[3]*2 * x + defaults[4]

    def f_diff_2(x):
        return defaults[0]*20 * x**3 + defaults[1]*12 * x**2 + defaults[2]*6 * x + defaults[3]*2
        
    def f_diff_3(x):
        return defaults[0]*60 * x**2 + defaults[1]*24 * x + defaults[2]*6
        
    # print((" {0}  ||   {1}    ||    {2}   || {3}  ||   {4}   ||   {5}").format("Итерация", "u0", "F'(u0)", "F''(u0)", "h = f'(u0) / f''(u0)", "|f'(u0)|"))
    i = 0
    while abs(f_diff_1(u0)) >= eps:
        i += 1
        print(abs(f_diff_1(u0)))
        # print(("     {0:.0f}     || {1:.4f} || {2:.4f} || {3:.4f} || {4:.4f} || {5:.4f}").format(i, u0, f_diff_1(u0), f_diff_2(u0), f_diff_1(u0)/f_diff_2(u0), abs(f_diff_1(u0))))
        if f_diff_2(u0) == 0:
            u0 += 0.1
            continue
        u0 = u0 - f_diff_1(u0)/f_diff_2(u0)
    return u0, f(u0), i

newton(0.5, [1, 0, 3, 0, 3, 1], 0.01)

## Общая (объединение всех функций)

In [0]:
def print_result(result):
    if result == Results.BISECTION:
        
        print('=============================')
        print('Метод деления отрезка пополам')
        print('=============================\n')
        
        x_min, f_min, n = bisection_method(a, b, x, f, eps)
        
        print(('Количество итераций: {0}').format(n))
        print(('Точка минимума: [{0:.4f}, {1:.4f}]\n').format(x_min, f_min))
        
    elif result == Results.GOLDEN:
        
        print('=============================')
        print('Метод "золотого сечения"')
        print('=============================\n')
        
        x_min, f_min, n = golden_section(f, a, b, eps)
        
        print(('Количество итераций: {0}').format(n))
        print(('Точка минимума: [{0:.4f}, {1:.4f}]\n').format(x_min, f_min))
        
    elif result == Results.PARABOLA:
        
        print('=============================')
        print('Метод парабол')
        print('=============================\n')
        
        x_min, f_min, n = f_parabola(a, b, x, f, eps)
        
        print(('Количество итераций: {0}').format(n))
        print(('Точка минимума: [{0:.4f}, {1:.4f}]\n').format(x_min, f_min))
        
    elif result == Results.NEWTON:
        
        print('=============================')
        print('Метод Ньютона')
        print('=============================\n')
        
        x_min, f_min, n = newton(u0, f.__defaults__, eps)
        
        print(('Начальная точка u0: {0}').format(u0))
        print(('Количество итераций: {0}').format(n))
        print(('Точка минимума: [{0:.4f}, {1:.4f}]\n').format(x_min, f_min))

In [0]:
Results = Enum('Results', 'BISECTION GOLDEN PARABOLA NEWTON')

def f(x):
    default = f.__defaults__
    y = default[0]*x**5 + default[1]*x**4 + default[2]*x**3 + default[3]*x**2 + default[4]*x + default[5]
    return y

def interval_check(a, b):
    x = np.linspace(a, b, 50000)
    default = f.__defaults__
    y = default[0]*20*x**3 + default[1]*12*x**2 + default[2]*6*x + default[3]*2
    if not np.all(y>=0):
        print('Промежуток унимодальности задан неверно')
        sys.exit(-1)
    return x
    
''' по дефолту значения функции 10 варианта, свои можно задать через
    f.__defaults__  , суть в том, что в Python значения по умолчанию
    иницилизируются единожды'''
    
f.__defaults__ = (1, 0, 3, 0, 3, 1) #задавайте тут свои параметры

a = 0.0001
b = 5
u0 = 0.05 #точка для метода Ньютона
eps = 0.001
x = interval_check(a, b)

print_result(Results.BISECTION)
print_result(Results.GOLDEN)
print_result(Results.PARABOLA)
print_result(Results.NEWTON)

# plt.plot(x, f(x), label='stock plot')
# plt.legend()

# plt.plot(x_min, f_min, 'ro')
# plt.ylim(-10, 10)


Функция для нахождения корней (вдруг нужно будет искать отрезки унимодальности)

In [0]:
def korni(f):
    x = symbols('x', real=True)
    print(solve(f(x), x))
    
def f_diff_2(x):
    defaults = f.__defaults__
    return defaults[0]*20 * x**3 + defaults[1]*12 * x**2 + defaults[2]*6 * x + defaults[3]*2
    

# Метод дробления шага

In [0]:
eps = float(input("Enter eps: "))
alpha = float(input("Enter alpha: "))
u0 = np.array([float(input('first coordinate')), float(input('second coordinate'))])
def f(u1, u2):
    return 4*u1*u2 + u1**2 + u2**2

def f_u1(u1, u2):
    return 4*u2 + 2*u1

def f_u2(u1, u2):
    return 4*u1 + 2*u2

def module_grad(grad):
    return sqrt(grad[0] ** 2 + grad[1]**2)

def grad(u1, u2):
    return np.array([f_u1(u1, u2), f_u2(u1, u2)])

k = 0
while module_grad(grad(*u0)) >= eps:
    k += 1
    u1 = u0 - alpha * grad(*u0)
    # print(f'f(u0), u0: {f(*u0)}, {u0}; f(u1), u1: {f(*u1)}, {u1}, grad: {grad(*u0)}')
    if f(*u1) < f(*u0):
        u0 = u1
    else:
        alpha /= 2

print(f'f(u0) = {f(*u0)}, u0 = {u0}, count of  k = {k}')

In [0]:
x = np.linspace(-100, 100, 1000000)
y = np.linspace(-100, 100, 1000000)
f(x, y).min(), np.argmin(f(x, y))

# Метод градиентного спуска

In [0]:
def f(u1, u2):
    return 4*u1*u2 + u1**2 + u2**2
    
def f_u1(u1, u2):
    return 4*u2 + 2*u1
    
def f_u2(u1, u2):
     return 4*u1 + 2*u2

def module_grad(grad):
    return sqrt(grad[0] ** 2 + grad[1]**2)
    
    
eps = float(input("Enter eps: "))
u0 = np.array([float(input('first coordinate: ')), float(input('second coordinate: '))])
def grad(u1, u2):
    return np.array([f_u1(u1, u2), f_u2(u1, u2)])
k = 0
def f_help(alpha):
    return f(*(u0 - alpha * grad(*u0)))
n = 0
# print(module_grad(grad(*u0)))
alpha = 1
while(module_grad(grad(*u0)) >= eps and alpha!=0):
    k += 1
    alpha, _, n = bisection_method(-100, 100, 1, f_help)
    u0 -= alpha * np.array(grad(*u0))
    k += n


print(f'f(u0) = {f(*u0)}, u0 = {u0}, count of  k = {k}')

In [0]:
f(-4.43962931e+152,  8.83450685e+152)

# Метод Ньютона

In [0]:
def f(u1, u2):
    return 4*u1*u2 + u1**2 + u2**2
    
def f_u1(u1, u2):
    return 4*u2 + 2*u1
    
def f_u2(u1, u2):
     return 4*u1 + 2*u2

def module_grad(grad):
    return sqrt(grad[0] ** 2 + grad[1]**2)
    
    
eps = float(input("Enter eps: "))
u0 = np.array([float(input('first coordinate: ')), float(input('second coordinate: '))])
def grad(u1, u2):
    return np.array([f_u1(u1, u2), f_u2(u1, u2)])
k = 0
def f_help(alpha):
    return f(*(u0 - alpha * grad(*u0)))
while(module_grad(grad(*u0)) >= eps and k<4):
    k += 1
    # print(f'f(u0) = {f(*u0)}, u0 = {u0}; grad={grad(*u0)}')
    # print(f'alpha={alpha}')
    # print('result:', np.array([grad(*u0)[0]*(-1/6) + grad(*u0)[1]*(1/3), grad(*u0)[0]*(1/3) + grad(*u0)[1]*(-1/6)]))
    u0 -= np.array([grad(*u0)[0]*(-1/6) + grad(*u0)[1]*(1/3), grad(*u0)[0]*(1/3) + grad(*u0)[1]*(-1/6)])


print(f'f(u0) = {f(*u0)}, u0 = {u0}, count of  k = {k}')

# Метод Штрафных функций

In [8]:
import numpy as np
from math import sqrt
def f(u1, u2, r):
    return (u1-3)**2 + (u2-2)**2 + 1/r * (u1+u2-4)**2
    
def f_u1(u1, u2, r):
    return 2*(u1-3) + 2*(u1+u2-4)*1/r
    
def f_u2(u1, u2, r):
    return 2*(u2-2) + 2*(u1+u2-4) * 1/r

def module_grad(grad):
    return sqrt(grad[0] ** 2 + grad[1]**2)
    
    

def grad(u1, u2, r):
    return np.array([f_u1(u1, u2, r), f_u2(u1, u2, r)])

def DG(u0, r, eps=0.0001):
    k = 0
    def f_help(alpha, r):
        return f(*(u0 - alpha * grad(*u0, r)), r)
    n = 0
    # print(module_grad(grad(*u0)))
    alpha = 1
    while(module_grad(grad(*u0, r)) >= 0.001 and alpha!=0):
        k += 1
        alpha, _, n = bisection_method(-100, 100, 1, f_help, r)
        u0 -= alpha * np.array(grad(*u0, r))
        print(grad(*u0, r), r)
        k += n

    return k, u0


r = 1
count = 0
u0 = [2, 4]
while(r>=0.001):
    count += 1
    print('u0:', u0)
    k, u0 = DG(u0, r)
    count += k
    r/=10
count, u0


u0: [2, 4]
[-2.86144776  0.70782835] 1
[0.40994528 1.65093652] 1
[-0.59103779  0.14720228] 1
[0.08560251 0.34115259] 1
[-0.12196414  0.03005733] 1
[0.0172203  0.07030817] 1
[-0.02537542  0.00612445] 1
[0.00353067 0.01462696] 1
[-0.0052888   0.00129522] 1
[0.00075205 0.00305128] 1
[-0.00109343  0.00027433] 1
[0.00015705 0.00063052] 1
u0: [2.66661393 1.66685066]
[-0.05646153 -0.05601082] 0.1
[0.00031232 0.00074137] 0.1
u0: [2.52371481 1.52392933]
[-0.63020042 -0.62977366] 0.01
[0.04606363 0.04648811] 0.01
[-0.00361031 -0.0031881 ] 0.01
[3.97129405e-05 4.59666384e-04] 0.01
u0: [2.50238319 1.50259317]
[-4.71770769 -4.71728806] 0.001
[2.48421853 2.48463784] 0.001
[-1.30861145 -1.30819245] 0.001
[0.68884888 0.68926756] 0.001
[-0.36309554 -0.36267719] 0.001
[0.19090185 0.19131989] 0.001
[-0.10085569 -0.10043798] 0.001
[0.05279606 0.05321346] 0.001
[-0.02812302 -0.02770594] 0.001
[0.01449253 0.01490929] 0.001
[-0.00795033 -0.00753388] 0.001
[0.00386925 0.00428537] 0.001
[-0.00235518 -0.0019393

(631, array([2.50014593, 1.50035352]))

# Метод Барьерных функций

In [ ]:
import numpy as np
from math import sqrt
def f(u1, u2, r):
    return (u1-3)**2 + (u2-2)**2 + 1/r * (u1+u2-4)**2
    
def f_u1(u1, u2, r):
    return 2*(u1-3) + 2*(u1+u2-4)*1/r
    
def f_u2(u1, u2, r):
    return 2*(u2-2) + 2*(u1+u2-4) * 1/r

def module_grad(grad):
    return sqrt(grad[0] ** 2 + grad[1]**2)
    
    

def grad(u1, u2, r):
    return np.array([f_u1(u1, u2, r), f_u2(u1, u2, r)])

def DG(u0, r, eps=0.0001):
    k = 0
    def f_help(alpha, r):
        return f(*(u0 - alpha * grad(*u0, r)), r)
    n = 0
    # print(module_grad(grad(*u0)))
    alpha = 1
    while(module_grad(grad(*u0, r)) >= 0.001 and alpha!=0):
        k += 1
        alpha, _, n = bisection_method(-100, 100, 1, f_help, r)
        u0 -= alpha * np.array(grad(*u0, r))
        print(grad(*u0, r), r)
        k += n

    return k, u0


r = 1
count = 0
u0 = [2, 4]
while(r>=0.001):
    count += 1
    print('u0:', u0)
    k, u0 = DG(u0, r)
    count += k
    r/=10
count, u0


0.00343094183984771